# pandas

## Exercise 1

(a) s1 with the values “apple”, “banana”, “cherry” with the name “fruit”.

(b) s2 with the values 3.14, 2.71, 1.61 with the name “transcendentals”.

In [ ]:
import pandas as pd

s1 = pd.Series(["apple", "banana", "cherry"], name="fruit")
s2 = pd.Series([3.14, 2.71, 1.61], name="transcendentals")

s1

In [ ]:
s2

## Exercise 2

In [ ]:
import numpy as np

d1 = pd.DataFrame(
    [[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6]],
    index=["a", "b", "c", "d"],
    columns=["a", "b", "c"],
)
d2 = pd.DataFrame(
    [[1, 2, 3], [1, 2, 3], [1, 2, 3]], columns=["Albania", "Belgium", "Canada"]
)
d3 = pd.DataFrame(np.arange(9).reshape((3, 3)))
d1

In [ ]:
d2

In [ ]:
d3

## Exercise 3

In [ ]:
d1["b"]

Using double brackets returns a DataFrame `[[ ]]`

In [ ]:
d1[["b"]]

In [ ]:
d2.Albania

## Exercise 4

In [ ]:
d2[["Albania", "Belgium"]]

## Exercise 5


In [ ]:
s3 = pd.Series([4, 4, 4], name="Denmark")

In [ ]:
d2["Denmark"] = s3
d2

In [ ]:
d2.pop("Denmark")
d2

In [ ]:
d2 = pd.concat([d2, s3], axis=1)
d2

## Exercise 6


In [ ]:
d2c = d2.copy()
d2c

In [ ]:
del d2c["Canada"]
d2c

In [ ]:
s4 = d2c.pop("Denmark")
s4

In [ ]:
d2c

## Exercise 7

In [ ]:
d4 = pd.concat([s1, s2], axis=1)
d4

In [ ]:
d4 = pd.DataFrame({s1.name: s1, s2.name: s2})
d4

## Exercise 8


In [ ]:
data = pd.read_csv("problem8.csv", index_col="Date", parse_dates=True)
data.loc["2020"].tail()

In [ ]:
data.loc["2019-12"].tail()

In [ ]:
data.loc["2019-03-26"]

In [ ]:
data.sort_index(ascending=False)

## Exercise 9

In [ ]:
data.shape

In [ ]:
nona = data.dropna()
nona.head()

In [ ]:
nona.shape

In [ ]:
filled = data.ffill()
filled.tail()

In [ ]:
data.dropna(axis=1)

In [ ]:
data.fillna(data.mean())

## Exercise 10
Convert the prices using the data in problem 8 to returns, store as rets, and:

(a) Select all returns > 1%.

(b) Select all returns outside of $\pm2\%$.

In [ ]:
rets = data.pct_change().dropna()

rets.loc[rets.AXJO > 0.01, "AXJO"]

In [ ]:
rets.loc[rets.GSPC > 0.01, "GSPC"]

In [ ]:
rets.loc[(rets.AXJO > 0.02) | (rets.AXJO < -0.02), "AXJO"]

## Exercise 11

In [ ]:
def trunc(val):
    return max(min(val, 0.02), -0.02)


rets.applymap(trunc)

## Exercise 12

In [ ]:
data.resample("M").max() / data.resample("M").min() - 1

## Exercise 13

In [ ]:
rets["month"] = rets.index.month
rets.groupby("month").mean()

## Exercise 14
Create a Categorical for extreme positive (upper 1%) and extreme negative returns (lower 1%) and non-extreme returns everything else.

In [ ]:
axjo = rets.AXJO.copy()
bounds = axjo.quantile([0.01, 0.99])
axjo.loc[axjo < bounds.loc[0.01]] = -1
axjo.loc[axjo > bounds.loc[0.99]] = 1
axjo.loc[~((axjo < bounds.loc[0.01]) | (axjo > bounds.loc[0.99]))] = 0
cats = axjo.astype("category")
cats

## Exercise 15

In [ ]:
pd.get_dummies(cats)

## Exercise 16

In [ ]:
rets.to_hdf("rets.hdf", "rets")
rets.to_stata("rets.dta")
rets.to_excel("rets.xlsx")
rets.to_csv("rets.csv")

## Exercise 17

* HDF needs no changes
* The rest need the `index_col` set to `"Date"`
* CSV needs to be told to `parse_dates`

In [ ]:
pd.read_hdf("rets.hdf", "rets")

In [ ]:
pd.read_stata("rets.dta", index_col="Date")

In [ ]:
pd.read_excel("rets.xlsx", index_col="Date")

In [ ]:
pd.read_csv("rets.csv", index_col="Date", parse_dates=True)

## Exercise 18
Plot the standard deviation of returns by month along side a 22-day rolling standard deviation.

In [ ]:
del rets["month"]

In [ ]:
# Restore the returns
rets = data.pct_change().dropna()
# Resample to monthly
monthly = rets.resample("M").std()
# Roll
rolling = rets.rolling(22).std()
# Rename columns
monthly.columns = [f"{c}_monthly" for c in monthly.columns]
rolling.columns = [f"{c}_rolling" for c in rolling.columns]
# concat
final = pd.concat([monthly, rolling], axis=1)
# Sort the index to ascend
final = final.sort_index()
# Forward fill and drop missing
final = final.ffill().dropna()
final.plot()